In [1]:
import pydicom as dicom
import numpy as np
import scipy 
import os

In [2]:
def get_contour_file(path):
    
    # handles `/` missing
    if path[-1] != '/': path += '/'
    
    # get .dcm contour file
    fpaths = [path + f for f in os.listdir(path) if '.dcm' in f]
    n = 0
    for fpath in fpaths:
        f = dicom.read_file(fpath)
        if 'ROIContourSequence' in dir(f):
            contour_file = fpath.split('/')[-1]
            n += 1
    if n > 1: warnings.warn("Many Files")
    if contour_file is None: print("No contour files in the selected directory")
    return contour_file

In [3]:
def print_roi_names(path, file):
    
    # checks if the path contains '/' and reads the file
    if path[-1] != '/': path += '/'
    d=dicom.dcmread(path + file)
    
    n = len(d.StructureSetROISequence)
    print('Il y a', n, 'structures dans le fichier:')

    for i in range(n):
        print('  ', i, ':', d.StructureSetROISequence[i].ROIName)

In [4]:
def plotcontour(img_arr, contour_arr, figsize=(20, 20)):
    """
    Shows 2d MR img with contour
    Inputs
        img_arr: 2d np.array image array with pixel intensities
        contour_arr: 2d np.array contour array with pixels of 1 and 0
    """

    masked_contour_arr = np.ma.masked_where(contour_arr == 0, contour_arr)
    plt.figure(figsize=figsize)
    plt.subplot(1, 2, 1)
    plt.imshow(img_arr, cmap='hot', interpolation='none')
    plt.subplot(1, 2, 2)
    plt.imshow(img_arr, cmap='hot', interpolation='none')
    plt.imshow(masked_contour_arr, cmap='cool', interpolation='none', alpha=0.6)
    plt.show()

In [5]:
m_path='imgs/'
contour_file = get_contour_file(m_path)
print_roi_names(m_path, contour_file)

Il y a 2 structures dans le fichier:
   0 : External
   1 : ext+1.6cm


In [8]:
a=dicom.dcmread(m_path + contour_file)
print(a.ROIContourSequence)
a.ROIContourSequence[1].ContourSequence



[(0021, 0010) Private Creator                     LO: 'NUCLETRON'
(0021, 1000) Private tag data                    UI: 1.3.6.1.4.1.2452.6.758510512.1171258812.3936947857.2645619669
(3006, 002a) ROI Display Color                   IS: [255, 173, 91]
(3006, 0040)  Contour Sequence   183 item(s) ---- 
   (3006, 0016)  Contour Image Sequence   1 item(s) ---- 
      (0008, 1150) Referenced SOP Class UID            UI: CT Image Storage
      (0008, 1155) Referenced SOP Instance UID         UI: 1.3.6.1.4.1.2452.6.1153380461.1330345744.3177375909.1484819030
      ---------
   (3006, 0042) Contour Geometric Type              CS: 'CLOSED_PLANAR'
   (3006, 0046) Number of Contour Points            IS: "199"
   (3006, 0048) Contour Number                      IS: "1"
   (3006, 0050) Contour Data                        DS: Array of 597 elements
   ---------
   (3006, 0016)  Contour Image Sequence   1 item(s) ---- 
      (0008, 1150) Referenced SOP Class UID            UI: CT Image Storage
      (00

<Sequence, length 208>

In [ ]:
#verification contour sur CT
RIO=1
contour=108
g=int(contour)
f=int(RIO)

d=np.zeros((512,512))
d1=np.zeros((512,512))

for u in range (g-2,g+2):
     contour_dataset=a.ROIContourSequence[f].ContourSequence[u]
     img_ID = contour_dataset.ContourImageSequence[0].ReferencedSOPInstanceUID
     img = dicom.read_file(path +'CT'+ img_ID + '.dcm')
     p=img.pixel_array
     x_spacing, y_spacing = float(img.PixelSpacing[0]), float(img.PixelSpacing[1])
     origin_x, origin_y, _ = img.ImagePositionPatient 
     
     c1=contour_dataset.ContourData
     points=int(a.ROIContourSequence[f].ContourSequence[u].NumberOfContourPoints) 
     print(points)
     for i in range (0,points-1):
     
          y1=np.int((around((c1[(i*3)+1]))-origin_y)/ y_spacing)
          x1=np.int((around((c1[(i*3)]))-origin_x)/ x_spacing)
          y2=np.int((around((c1[((i+1)*3)+1]))-origin_y)/ y_spacing)
          x2=np.int((around((c1[((i+1)*3)]))-origin_x)/ x_spacing)
          d[y1,x1]=1000
          if x1 != x2 :
              a=((y1-y2)/(x1-x2))
              b=0.5*((y1+y2)-(a*(x1+x2)))
         
              if x1<x2 :
                  for k in range(x1-1,x2+1):
                    y=np.int(around((a*k)+b))
                    d1[y,k]=1000
             
              if x1>x2:
                  for e in range(x2-1,x1+1):
                      y=np.int(around(((a*e+b))))
                      d1[y,e]=1000
              if x1==x2:
            
                
                     d1[y1,x1]=1000
                     
                    
          if y1 != y2 :
                
              a=((x1-x2)/(y1-y2))
              b=0.5*((x1+x2)-(a*(y1+y2)))
         
              if y1<y2 :
                  for k in range(y1-1,y2+1):
                    x=np.int(around((a*k)+b))
                    d1[k,x]=1000
             
              if y1>y2:
                  for e in range(y2-1,y1+1):
                      x=np.int(around(((a*e+b))))
                      d1[e,x]=1000
              if y1==y2:
            
                
                     d1[y1,x1]=1000
     i=0               
     y1=np.int((around((c1[(i*3)+1]))-origin_y)/ y_spacing)
     x1=np.int((around((c1[(i*3)]))-origin_x)/ x_spacing)
     i=points-2
     y2=np.int((around((c1[((i+1)*3)+1]))-origin_y)/ y_spacing)
     x2=np.int((around((c1[((i+1)*3)]))-origin_x)/ x_spacing)
     d1[y1,x1]=1000
     if x1 != x2 :
              a=((y1-y2)/(x1-x2))
              b=0.5*((y1+y2)-(a*(x1+x2)))
         
              if x1<x2 :
                  for k in range(x1-1,x2+1):
                    y=np.int(around((a*k)+b))
                    d1[y,k]=1000
             
              if x1>x2:
                  for e in range(x2-1,x1+1):
                      y=np.int(around(((a*e+b))))
                      d1[y,e]=1000
              if x1==x2:
            
                
                     d1[y1,x1]=1000
                     
                    
     if y1 != y2 :
                
              a=((x1-x2)/(y1-y2))
              b=0.5*((x1+x2)-(a*(y1+y2)))
         
              if y1<y2 :
                  for k in range(y1-1,y2+1):
                    x=np.int(around((a*k)+b))
                    d1[k,x]=1000
             
              if y1>y2:
                  for e in range(y2-1,y1+1):
                      x=np.int(around(((a*e+b))))
                      d1[e,x]=1000
              if y1==y2:
            
                
                     d1[y1,x1]=1000
d2=d1+d       
d4=np.zeros((512,512))
for i in range (0,512):
    for j in range(0,512):
        if d2[i,j]>0:
            
            for a in range (j,512):
                 if d2[i,a]>0:
                    for e in range (j,a):
                        d4[i,e]=1000
d3=np.zeros((512,512))
for j in range (0,512):
    for i in range(0,512):
        if d2[i,j]>0:
            
            for a in range (i,512):
                 if d2[a,j]>0:
                    for e in range (i,a):
                        d3[e,j]=1000
d5=np.zeros((512,512))                        
for i in range (0,512):
    for j in range(0,512):
        if d3[i,j]>0 and d4[i,j]>0:
            d5[i,j]=1000
        
plot2dcontour(p,d5)          

In [ ]:
roi_names(contour_file)
a=dicom.dcmread(path + '/' + contour_file)

In [ ]:
a=dicom.dcmread(path + '/' + contour_file)
a.ROIContourSequence[1].ContourSequence

In [ ]:
RIO=1
N_contours=208
mm=np.zeros((128,128,128))


   
for pp in range (0,N_contours-1):
     g=int(pp)
     f=int(RIO)

     d=np.zeros((128,128))
     d1=np.zeros((128,128))
     for u in range (g-2,g+1):
         
         a=dicom.dcmread(path + '/' + contour_file)
         contour_dataset=a.ROIContourSequence[f].ContourSequence[u]
   
         img = dicom.read_file('imgs/spect.DCM')
         p=img.pixel_array
         x_spacing, y_spacing,z_spacing = float(img.PixelSpacing[0]),float(img.PixelSpacing[0]), float(img.SliceThickness)
         origin_x, origin_y,origin_z = img.DetectorInformationSequence[0].ImagePositionPatient
         c1=contour_dataset.ContourData
         nn=128-(around((c1[2]+origin_z)/z_spacing))
         v_pixel=x_spacing*y_spacing*z_spacing
         points=int(a.ROIContourSequence[f].ContourSequence[u].NumberOfContourPoints)  
         for i in range (0,points-1):
     
               y1=np.int(((c1[(i*3)+1])-origin_y)/ y_spacing)
               x1=np.int((around((c1[(i*3)]))-origin_x)/ x_spacing)
               y2=np.int((around((c1[((i+1)*3)+1]))-origin_y)/ y_spacing)
               x2=np.int((around((c1[((i+1)*3)]))-origin_x)/ x_spacing)
               d[y1,x1]=1000
               

               if x1 != x2 :
                    a=((y1-y2)/(x1-x2))
                    b=0.5*((y1+y2)-(a*(x1+x2)))
         
                    if x1<x2 :
                       for k in range(x1,x2+1):
                           y=np.int(around((a*k)+b))
                           d1[y,k]=1000
              
                    if x1>x2:
                       for e in range(x2,x1+1):
                           y=np.int(around(((a*e+b))))
                           d1[y,e]=1000
                    if x1==x2:
            
                
                           d1[y1,x1]=1000
                     
                    
               if y1 != y2 :
                
                   a=((x1-x2)/(y1-y2))
                   b=0.5*((x1+x2)-(a*(y1+y2)))
         
                   if y1<y2 :
                          for k in range(y1,y2+1):
                            x=np.int(around((a*k)+b))
                            d1[k,x]=1000
             
                   if y1>y2:
                          for e in range(y2,y1+1):
                              x=np.int(around(((a*e+b))))
                              d1[e,x]=1000
                   if y1==y2:
            
                
                     d1[y1,x1]=1000
                    
         i=0               
         y1=np.int((around((c1[(i*3)+1]))-origin_y)/ y_spacing)
         x1=np.int((around((c1[(i*3)]))-origin_x)/ x_spacing)
         i=points-2
         y2=np.int((around((c1[((i+1)*3)+1]))-origin_y)/ y_spacing)
         x2=np.int((around((c1[((i+1)*3)]))-origin_x)/ x_spacing)
         d1[y1,x1]=1000
         if x1 != x2 :
                  a=((y1-y2)/(x1-x2))
                  b=0.5*((y1+y2)-(a*(x1+x2)))
         
                  if x1<x2 :
                      for k in range(x1,x2+1):
                        y=np.int(around((a*k)+b))
                        d1[y,k]=1000
             
                  if x1>x2:
                      for e in range(x2,x1+1):
                          y=np.int(around(((a*e+b))))
                          d1[y,e]=1000
                  if x1==x2:
            
                
                         d1[y1,x1]=1000
                     
                    
         if y1 != y2 :
                
                  a=((x1-x2)/(y1-y2))
                  b=0.5*((x1+x2)-(a*(y1+y2)))
         
                  if y1<y2 :
                      for k in range(y1,y2+1):
                        x=np.int(around((a*k)+b))
                        d1[k,x]=1000
             
                  if y1>y2:
                      for e in range(y2,y1+1):
                          x=np.int(around(((a*e+b))))
                          d1[e,x]=1000
                  if y1==y2:
            
                
                         d1[y1,x1]=1000
     d2=d1+d       
     d4=np.zeros((128,128))
     for i in range (0,128):
         for j in range(0,128):
             if d2[i,j]>0:
            
                 for a in range (j,128):
                      if d2[i,a]>0:
                         for e in range (j,a):
                             d4[i,e]=1000
     d3=np.zeros((128,128))
     for j in range (0,128):
         for i in range(0,128):
             if d2[i,j]>0:
            
                 for a in range (i,128):
                      if d2[a,j]>0:
                         for e in range (i,a):
                             d3[e,j]=1000
     d5=np.zeros((128,128))                        
     for i in range (0,128):
         for j in range(0,128):
             if d3[i,j]>0 and d4[i,j]>0:
                 d5[i,j]=1000
     
     mm[int(nn)]=d5

In [ ]:
i=50
plotcontour(p[i],mm[i])   

In [ ]:
for k in range (0,128):
           if np.sum(mm[k])>0:
             mm[k-1]=mm[k]
             #mm[k-2]=mm[k]
             break
for k in range (0,127):
           if np.sum(mm[127-k])>0:
              mm[127-k+1]=mm[127-k]
             #mm[k-2]=mm[k]
              break
mn=np.zeros((128,128,128))

for k in range (0,128):
    for i in range(0,128):
        for j in range(0,128):
            mn[k,i,j]=mm[i,k,j]
            
sp=np.zeros((128,128,128))
x=dicom.dcmread('imgs/spect.DCM')
p=x.pixel_array
for k in range (0,128):
    for i in range(0,128):
        for j in range(0,128):
            sp[k,i,j]=p[i,k,j]
i=65          
plot2dcontour(sp[i],mn[i])

In [ ]:
N=0
pc=np.zeros((128,128,128))
for k in range (0,128):
    for i in range(0,128):
        for j in range(0,128):
             if mm[k,i,j]>0 :
                  pc[k,i,j]=p[k,i,j]
                  N=N+1
                    
v_f=(N*v_pixel) *10**(-3)                
print('le volume est de=',v_f  )                 
cp=np.sum(pc)
hour=int(img.AcquisitionTime[0:2])-14
min=int(img.AcquisitionTime[2:4])-59
t_s1=(23+hour)*60+min+10        #temps de comencent d'acqusition detecteur1  (min)
t_aq=int(img.RotationInformationSequence[0].ActualFrameDuration)*0.5/10**3         #duré d'acqusition (min)
t_p=792     #periode physique de 123-I (min)
#----------------------------------------------------------------------------------
A_inj_s1 =0.001093*16+(v_f+(-16))*0.000298   # l'activité de la suringue pliene (mCi)
t_s11=1       #temps de mesure  l'activité de la suringue pliene (min)
A_ss1=0   # l'activité de la suringue vide (mCi)
t_ss1=0       #temps de mesure  l'activité de la suringue vide (min)

A_ss1=(A_ss1 * exp((log(2)*t_ss1)/t_p))
A_inj_s1=(A_inj_s1 * exp((log(2)*t_s11)/t_p))
#correction de l'activité exact injecté  .
A_s1=A_inj_s1-A_ss1
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
A_inj_s2 =0.001093*8 +(v_f+(-8))*0.000298   # l'activité de la suringue pliene (mCi)
t_s2=1       #temps de mesure  l'activité de la suringue pliene (min)
A_ss2=0   # l'activité de la suringue vide (mCi)
t_ss2=0       #temps de mesure  l'activité de la suringue vide (min)

A_ss2=(A_ss2 * exp((log(2)*t_ss2)/t_p))
A_inj_s2=(A_inj_s2 * exp((log(2)*t_s2)/t_p))
#correction de l'activité exact injecté  .
A_s2=A_inj_s2-A_ss2
#----------------------------------------------------------------------------------
A_inj_fond =2.04   # l'activité de la suringue pliene (mCi)
t_fond=1      #temps de mesure  l'activité de la suringue pliene (min)
A_sfond=0  # l'activité de la suringue vide (mCi)
t_sfond=4      #temps de mesure  l'activité de la suringue vide (min)

A_sfond=(A_sfond * exp((log(2)*t_sfond)/t_p))
A_inj_fond=(A_inj_fond * exp((log(2)*t_fond)/t_p))
#correction de l'activité exact injecté  .
A_fond=A_inj_fond-A_sfond
#----------------------------------------------------------------------------------


#----------------------------------------------------------------------------------






#A_total=A_pd+A_lf+A_lc+A_foie+A_fond+A_pg
A_total=A_fond
# l'activité au moment de commencement d'acquisition  .

# l'activité au moment de commencement d'acquisition  .
A_01=A_total * exp((-log(2)*t_s1)/t_p)


# l'activité moyenne entre le debut et la fin d'acquisition mCi .


# nombre de coups par seconde    
cps=cp/(t_aq*2*60)
# l'activité moyenne entre le debut et la fin d'acquisition (MBq)  .
A_m1=(A_01 *37*(t_p/log(2))* (1-exp(-log(2)*t_aq/t_p)))/t_aq

A_mesuré=cps/62.5
print('activité réelle =',A_m1)
print('activité mesuré',A_mesuré)
v=abs(A_mesuré-A_m1)*100/A_m1
print('variation relative',v,'%')

In [ ]:
img

In [ ]:
int(img.AcquisitionTime[0:2])-14